In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, PFNanoAODSchema
from coffea.nanoevents.methods import vector, candidate
import awkward as ak
import numpy as np
import uproot
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color=['blue', 'red', 'green', 'violet', 'darkorange', 'black', 'cyan', 'yellow'])
import mplhep as hep
plt.style.use(hep.style.CMS) # use CMS style for plotting

In [62]:
sam, fname = 'qcd', '/home/olympus/licq/data/calib/custom_nano/samples/20220415_fullGenPart/2017/mc/QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8/CustomNano_fullGenPart_RunIISummer20UL17MiniAODv2-106X_v9-v1/220415_194914/0000/out_NANO_2017_1.root'
sam, fname = 'hbb', '/home/olympus/licq/data/calib/custom_nano/samples/20220415_fullGenPart/2017/mc/ZH_HToBB_ZToNuNu_M-125_TuneCP5_13TeV-powheg-pythia8/CustomNano_fullGenPart_RunIISummer20UL17MiniAODv2-106X_v9-v1/220415_194343/0000//out_NANO_2017_1.root'
# sam, fname = 'hcc', '/home/olympus/licq/data/calib/custom_nano/samples/20220415_fullGenPart/2017/mc/ZH_HToCC_ZToNuNu_M-125_TuneCP5_13TeV-powheg-pythia8/CustomNano_fullGenPart_RunIISummer20UL17MiniAODv2-106X_v9-v1/220415_194401/0000/out_NANO_2017_1.root'
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()

mode = 'bb'

In [63]:
# fatjet match with two SV
fj_sv_comb = ak.cartesian([events.FatJet, events.SV], nested=True)
fj_sv_dr = fj_sv_comb['0'].delta_r(fj_sv_comb['1'])
fj_pass_nsv = ak.sum(fj_sv_dr < 0.8, axis=-1) >= 2

# each one of two subjets of a fatjet match with one SV
sj1 = events.SubJet[events.FatJet.subJetIdx1.mask[events.FatJet.subJetIdx1 >= 0]] # some fatjet has no subjet..
sj2 = events.SubJet[events.FatJet.subJetIdx2.mask[events.FatJet.subJetIdx2 >= 0]] # some fatjet has no subjet..
sj12_dr = sj1.delta_r(sj2)

sj1_sv_comb = ak.cartesian([sj1, events.SV], nested=True)
sj1_sv_dr = sj1_sv_comb['0'].delta_r(sj1_sv_comb['1'])
sj1_pass_nsv = ak.any(sj1_sv_dr < np.minimum(0.4, sj12_dr), axis=-1)
sj2_sv_comb = ak.cartesian([sj2, events.SV], nested=True)
sj2_sv_dr = sj2_sv_comb['0'].delta_r(sj2_sv_comb['1'])
sj2_pass_nsv = ak.any(sj2_sv_dr < np.minimum(0.4, sj12_dr), axis=-1)
# recover None entries (with no subjet)
sj1_pass_nsv = ak.fill_none(sj1_pass_nsv, False)
sj2_pass_nsv = ak.fill_none(sj2_pass_nsv, False)

if mode == 'bb':
    fj_pass_nhad = events.FatJet.nBHadrons >= 2
elif mode == 'cc':
    fj_pass_nhad = (events.FatJet.nBHadrons == 0) & (events.FatJet.nCHadrons >= 2)

fj_pass = fj_pass_nsv & fj_pass_nhad & sj1_pass_nsv & sj2_pass_nsv

# selection
evt = events[ak.sum(fj_pass, axis=-1) >= 1]
evt_fj_pass = fj_pass[ak.sum(fj_pass, axis=-1) >= 1]

In [64]:
def make_subplot(ax, tree, tree_fj_pass):
    # default plotting configuration
    f = lambda x: np.sqrt(x)/100
    alpha = 0.2

    def color_opt(color, fill=True):
        if fill:
            return {'facecolor': color}
        else:
            return {'edgecolor': color, 'linewidth': 2, 'fill': False, 'hatch': '///'}
    # jet
    fj = tree.FatJet
    for pt, eta, phi, ispass, t_bb, t_cc, t_qcd in zip(fj.pt, fj.eta, fj.phi, tree_fj_pass, fj.particleNetMD_Xbb, fj.particleNetMD_Xcc, fj.particleNetMD_QCD):
        edgecolor = 'red' if ispass else 'grey'
        # ax.add_patch(plt.Circle((eta, phi), radius=f(pt), clip_on=True, alpha=alpha, **color_opt('grey', fill=True)))
        ax.add_patch(plt.Circle((eta, phi), radius=0.8, clip_on=True, fill=False, edgecolor=edgecolor, linestyle=':'))
        ax.text(eta-0.6, phi+1.0, f'{t_bb:.3f}', fontsize=12, ha='center', va='center', color='purple')
        ax.text(eta, phi+1.0, f'{t_cc:.3f}', fontsize=12, ha='center', va='center', color='deepskyblue')
        ax.text(eta+0.6, phi+1.0, f'{t_qcd:.3f}', fontsize=12, ha='center', va='center', color='grey')
        ax.text(eta-0.3, phi+0.75, f'{(t_bb/(t_bb + t_qcd)):.3f}', fontsize=12, ha='center', va='center', color='purple')
        ax.text(eta+0.3, phi+0.75, f'{(t_cc/(t_cc + t_qcd)):.3f}', fontsize=12, ha='center', va='center', color='deepskyblue')
    # subjets
    sj = tree.SubJet
    for pt, eta, phi in zip(sj.pt, sj.eta, sj.phi):
        ax.add_patch(plt.Circle((eta, phi), radius=f(pt), clip_on=True, alpha=alpha, **color_opt('grey', fill=False)))
     # SV
    sv = tree.SV
    for pt, eta, phi in zip(sv.pt, sv.eta, sv.phi):
        ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 4, radius=f(pt)*10, clip_on=True, alpha=alpha, **color_opt('orange', fill=True)))
    # partons/hadrons
    genp = tree.GenPart
    for pt, eta, phi, pid in zip(genp.pt, genp.eta, genp.phi, genp.pdgId):
        pid = abs(pid)
        if (pid >= 400 and pid < 500) or (pid >= 4000 and pid < 5000):
            color = 'deepskyblue'
        elif (pid >= 500 and pid < 600) or (pid >= 5000 and pid < 6000):
            color = 'purple'
        else:
            continue
        ax.add_patch(plt.Circle((eta, phi), radius=f(pt), clip_on=True, alpha=alpha, **color_opt(color, fill=True)))

    ax.set_xlim(-4, 4); ax.set_ylim(-4, 4)
    ax.set_xlabel(r'$\eta$'); ax.set_ylabel(r'$\phi$')
    ax.set_aspect('equal')

from tqdm import trange
print(mode, sam)
for i in trange(30):
    fig, ax = plt.subplots(figsize=(10, 10))
    make_subplot(ax, evt[i], evt_fj_pass[i])
    plt.savefig(f"../plots/event_vis/{mode}/{sam}_{i:03d}.png")
    plt.close()

bb hbb


100%|██████████| 30/30 [00:10<00:00,  2.91it/s]
